# Testing merges 

In [ ]:
import numpy as np
import pandas as pd
import math
from siuba import *
from calitp import *
import intake
import data_prep
import fuzzymatcher
pd.options.display.max_rows = 250

In [ ]:
df_5311 = data_prep.load_grantprojects()
vehicles = data_prep.load_cleaned_vehiclesdata()
organizations = data_prep.load_cleaned_organizations_data()

In [ ]:
GTFS = data_prep.GTFS()

# Merge NTD with GTFS  --> vehicle_gtfs
<b> NOTE</b>:
I had to manually add NTD ID to the following agencies in the original CSV file called "cleaned organizations.csv"

* Butte County Association of Governments	90208
* City of Dixon	9402-91041
* City of Fairfield	90092
* City of Ridgecrest	9R02-91006
* City of Wasco	9R02-99426
* Glenn County	9R02-91088
* Mariposa County	9R02-91082
* Modoc Transportation Authority 	9R02-91008
* Palo Verde Valley Transit Agency	9R02-99454
* San Benito County Local Transportation Authority	9R02-91009
* San Joaquin Regional Transit District	90012
* Tuolumne County Transit Agency	9402-035


In [ ]:
#trying the iloc way 
organizations.loc[(organizations['name'] == 'Butte County Association of Governments'), "ntd_id"] = "90208"
vehicles.loc[(vehicles['agency'] == 'Butte County Association of Governments'), "ntd_id"] = "90208"

organizations.loc[(organizations['name'] == 'City of Fairfield'), "ntd_id"] = "90092"
vehicles.loc[(vehicles['agency'] == 'City of Fairfield, California, dba: Fairfield and Suisun Transit'), "ntd_id"] = "90092"

In [ ]:
#merging the 2 datasets together
vehicles_gtfs = pd.merge(vehicles, organizations,  how='left', on=['ntd_id'], indicator=True)

In [ ]:
vehicles_gtfs['_merge'].value_counts()

In [ ]:
vehicles_gtfs = vehicles_gtfs.drop(columns = ['_merge'])

In [ ]:
vehicles_gtfs.head(2)
vehicles_gtfs[(vehicles_gtfs.agency.str.contains("Butte", case= False))].head(2)

# Merging Black Cat with vehicles_gtfs

In [ ]:
right_on = ["agency"]
left_on = ["organization_name"]

In [ ]:
black_cat1 = fuzzymatcher.fuzzy_left_join(df_5311, vehicles_gtfs, left_on, right_on)

In [ ]:
unique_agencies = black_cat1[['best_match_score','organization_name','agency']]
unique_agencies = unique_agencies.drop_duplicates()

In [ ]:
#unique_agencies.sort_values('best_match_score')

In [ ]:
#stuff to filter out before cutting off at scores...
subset3 = ['County of Los Angeles - Department of Public Works', 'County of Sacramento Department of Transportation']

In [ ]:
#filter out a bunch the agencies above
black_cat2 = black_cat1[~black_cat1.organization_name.isin(subset3)]

In [ ]:
#filter out for any scores below 0.19
black_cat2 = black_cat2.loc[black_cat2['best_match_score'] > 0.19 ]

In [ ]:
#total agencies that matched over with decent scores
black_cat2['organization_name'].nunique()

## Walking back in the other agencies that were cut into a seperate dataframe

In [ ]:
#getting a list of agencies with bad scores
bad_fuzzy_matches = black_cat1.loc[black_cat1['best_match_score'] < 0.19 ]

In [ ]:
bad_fuzzy_matches = bad_fuzzy_matches[['organization_name','agency', 'ntd_id']]

In [ ]:
bad_fuzzy_matches = bad_fuzzy_matches.drop_duplicates()

In [ ]:
#making a manual crosswalk
#bad_fuzzy_matches.to_csv("./bad_fuzzy_matches.csv", index = False) 

In [ ]:
#reading crosswalk 3, what I manually inputted
crosswalk3 = pd.read_excel('corrected_bad_fuzzy_match.xlsx')

In [ ]:
#crosswalk on the left, I only want a dataframe with the poorly matched agencies
unmatched = crosswalk3.merge(df_5311, on=['organization_name'],  how='left')

In [ ]:
#now grabbing vehicles and GTFS info on the left
unmatched2 = unmatched.merge(vehicles_gtfs, on=['agency','ntd_id'],  how='left')

## R bind the badly matched columns back

In [ ]:
GTFS_Vehicles_BC = black_cat2.append(pd.DataFrame(data = unmatched2), ignore_index=True)

In [ ]:
GTFS_Vehicles_BC.shape

# Final Clean Up

## One last check to make sure all the names are there
<b> Still having issues with </b>
* Butte
* Fairfield
* Tuolumne 

<b> Inherently, aren't sure if these agencies correspond with any GTFS & NTD records</b> 

* Alpine County
* Fresno Council of Gov
* Greyhound
* County of LA
* City of Tehachapi
* Calaveras County Work
* County of Nevada Public Works
* County of Shasta Department of Public Works
* Glenn County Transportation Commission
* Tehama County Transit Agency

<b> Has GTFS but no NTD </b>
* Klamath Trinity Non-Emergency Transportationâ€

<b> Has NTD no GFS </b>
* Calaveras Transit Agency

In [ ]:
Names = GTFS_Vehicles_BC[['organization_name','agency','ntd_id','itp_id','gtfs_schedule_status']]
Names = Names.drop_duplicates()
#Names.sort_values('organization_name')

In [ ]:
Names.isna().sum()

## Generate a column that indicates that the rows that only appear in Black Cat 

In [ ]:
missing_list = Names[Names.isna().any(axis=1)]

In [ ]:
BC_only = missing_list[['organization_name']]
BC_only=BC_only.values.tolist()

In [ ]:
BC_only = ['County of Los Angeles - Department of Public Works','Greyhound Lines, Inc.','Alpine County Community Development',
'Fresno Council of Governments','Tehama County Transit Agency','Glenn County Transportation Commission','Stanislaus County Public Works - Transit Division','County Connection (Central Contra Costa Transit Authority)','County of Nevada Public Works, Transit Services Division',
'City of Tehachapi']

In [ ]:
len(BC_only)

In [ ]:
def flag_for_BC(row):
    if row.organization_name in BC_only:
        return '1'
    else: 
        return '0'                    

In [ ]:
GTFS_Vehicles_BC["Is_Agency_In_BC_Only_1_means_Yes"] = GTFS_Vehicles_BC.apply(lambda x: flag_for_BC(x), axis=1)

## Fill in Missing Values - Loops
* https://stackoverflow.com/questions/51097981/fill-column-of-a-dataframe-from-another-dataframe
* https://www.kite.com/python/answers/how-to-create-a-pandas-dataframe-from-columns-in-other-dataframes-in-python
* https://www.kite.com/python/answers/how-to-fill-a-pandas-dataframe-row-by-row-in-python
* https://www.geeksforgeeks.org/replace-values-of-a-dataframe-with-the-value-of-another-dataframe-in-pandas/

In [ ]:
v_gtfs_subset = vehicles_gtfs[['agency','ntd_id',
       'organization_type', 'reporter_type', 'primary_uza_population',
       'agency_voms', 'vehicle_type', 'years_old:', '_13_15', '_16_20',
       '_21_25', '_26_30', '_31_60', '_60+', 'total_vehicles',
       'average_age_of_fleet__in_years_', 'average_lifetime_miles_per_vehicle',
       '_0_9', '_10_12', 'vehicle_groups', 'name', 'itp_id',
       'gtfs_schedule_status']]

In [ ]:
subset_agencies = ['Butte County Association of Governments','City of Fairfield, California, dba: Fairfield and Suisun Transit']
subset1 = vehicles_gtfs[vehicles_gtfs.agency.isin(subset_agencies)]

In [ ]:
subset2 = GTFS_Vehicles_BC[GTFS_Vehicles_BC.agency.isin(subset_agencies)]
subset2 = subset2[['best_match_score', '__id_left', '__id_right', 'grant_fiscal_year',
       'funding_program', 'grant_number', 'project_year', 'organization_name',
       'upin', 'description', 'ali', 'contract_number', 'allocationamount',
       'encumbered_amount', 'expendedamount', 'activebalance',
       'closedoutbalance', 'project_status','agency']]

In [ ]:
subset3 = pd.merge(subset1, subset2,  how='outer', on=['agency'])

In [ ]:
subset3.shape

In [ ]:
GTFS_Vehicles_BC[(GTFS_Vehicles_BC.organization_name.str.contains("Dixon", case= False))].head(2)

#### Klamath has ITP data but no NTD data. Have to add it manually

In [ ]:
GTFS_Vehicles_BC.loc[(GTFS_Vehicles_BC['organization_name'] == 'Klamath Trinity Non-Emergency Transportation\u200b'), "itp_id"] = "436"
GTFS_Vehicles_BC.loc[(GTFS_Vehicles_BC['organization_name'] == 'Klamath Trinity Non-Emergency Transportation\u200b'), "gtfs_schedule_status"] = "needed"

## Replacing GTFS with more simple data
* This data frame didn't have NTD IDS, but they do have a more definitive GTFS status like "ok" or "needed."
* Using operator instead of provider and name because more gtfs_simple_status matches (502 verus 520 versus 830 ish)


In [ ]:
GTFS.head(1)
GTFS = GTFS.rename(columns = {'gtfs_schedule_status':'simple_GTFS_status'})

In [ ]:
GTFS_Vehicles_BC2 = pd.merge(GTFS_Vehicles_BC, GTFS,  how='left', left_on = ['organization_name'], right_on=['operator'], indicator=True)

In [ ]:
GTFS_Vehicles_BC2['gtfs_status_final'] = GTFS_Vehicles_BC2.apply(lambda x: x.gtfs_schedule_status if (str(x.simple_GTFS_status) == 'nan') else x.simple_GTFS_status	, axis=1)

## Final preview
* Deleted a few things: agency, state, legacy_id, years old
* LOTS of rows that look the same but the column "upin" makes them different. Do we need every year/contract that the agency has produced?

In [ ]:
GTFS_Vehicles_BC2 = GTFS_Vehicles_BC2.drop(columns=['best_match_score','__id_left','__id_right','state', 'legacy_ntd_id', 'years_old:','simple_GTFS_status','gtfs_schedule_status'])

In [ ]:
GTFS_Vehicles_BC2.duplicated().sum()

In [ ]:
#writing to GCS bucket
#GTFS_Vehicles_BC2.to_csv("gs://calitp-analytics-data/data-analyses/5311 /GTFS_Vehicles_BC.csv", index= False)